In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

/upb/users/s/ssmirnov/profiles/unix/imt/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.418767
Minibatch accuracy: 0.0%
Validation accuracy: 10.1%
Minibatch loss at step 50: 1.717316
Minibatch accuracy: 37.5%
Validation accuracy: 53.9%
Minibatch loss at step 100: 0.993146
Minibatch accuracy: 75.0%
Validation accuracy: 67.9%
Minibatch loss at step 150: 1.062768
Minibatch accuracy: 75.0%
Validation accuracy: 74.9%
Minibatch loss at step 200: 0.810700
Minibatch accuracy: 81.2%
Validation accuracy: 78.6%
Minibatch loss at step 250: 0.800039
Minibatch accuracy: 87.5%
Validation accuracy: 78.3%
Minibatch loss at step 300: 1.399895
Minibatch accuracy: 43.8%
Validation accuracy: 78.7%
Minibatch loss at step 350: 0.456779
Minibatch accuracy: 81.2%
Validation accuracy: 79.7%
Minibatch loss at step 400: 1.164904
Minibatch accuracy: 68.8%
Validation accuracy: 72.1%
Minibatch loss at step 450: 0.713285
Minibatch accuracy: 75.0%
Validation accuracy: 80.3%
Minibatch loss at step 500: 1.061420
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Mi

# Problem 1
The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (nn.max_pool()) of stride 2 and kernel size 2.



In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    
    # Input
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
  
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    def model(data):
        conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
        pool_1 = tf.nn.max_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
        pool_2 = tf.nn.max_pool(hidden_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden_3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden_3, layer4_weights) + layer4_biases
  
  # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.985262
Minibatch accuracy: 6.2%
Validation accuracy: 9.7%
Minibatch loss at step 50: 1.990104
Minibatch accuracy: 25.0%
Validation accuracy: 35.4%
Minibatch loss at step 100: 1.382714
Minibatch accuracy: 56.2%
Validation accuracy: 60.0%
Minibatch loss at step 150: 1.165500
Minibatch accuracy: 56.2%
Validation accuracy: 69.7%
Minibatch loss at step 200: 0.999157
Minibatch accuracy: 81.2%
Validation accuracy: 78.4%
Minibatch loss at step 250: 0.804208
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 300: 1.230048
Minibatch accuracy: 68.8%
Validation accuracy: 77.3%
Minibatch loss at step 350: 0.526594
Minibatch accuracy: 87.5%
Validation accuracy: 81.1%
Minibatch loss at step 400: 1.104328
Minibatch accuracy: 68.8%
Validation accuracy: 72.8%
Minibatch loss at step 450: 0.746298
Minibatch accuracy: 75.0%
Validation accuracy: 81.3%
Minibatch loss at step 500: 0.828754
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Min

# Problem 2
Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.


Lets start with just regular CNN. After that we will compare the perfomance with CNN + leaning rate decay and dropout. The LeNet can be roughly described as follows:
input -> conv_1 -> pool_1 -> conv_2 -> pool_2 -> fc_3 -> output

In [12]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    
    # Input
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    layer3_weights = tf.Variable(tf.truncated_normal(
        [size3 * size3 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model our CNN
    def model(data):
        
        # C1 input 28 x 28
        conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
        hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
        # S1 input 24 x 24
        pool_1 = tf.nn.max_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        
        # C2 input 12 x 12
        conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='VALID')
        hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
        # S2 input 8 x 8
        pool_2 = tf.nn.max_pool(hidden_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        
        # FC3 input 4 x 4
        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden_3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden_3, layer4_weights) + layer4_biases
  
  # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [14]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.875572
Minibatch accuracy: 6.2%
Validation accuracy: 8.6%
Minibatch loss at step 50: 1.769680
Minibatch accuracy: 43.8%
Validation accuracy: 46.3%
Minibatch loss at step 100: 1.076891
Minibatch accuracy: 62.5%
Validation accuracy: 60.1%
Minibatch loss at step 150: 1.530651
Minibatch accuracy: 50.0%
Validation accuracy: 69.6%
Minibatch loss at step 200: 1.153349
Minibatch accuracy: 81.2%
Validation accuracy: 75.0%
Minibatch loss at step 250: 0.857016
Minibatch accuracy: 68.8%
Validation accuracy: 75.0%
Minibatch loss at step 300: 1.650831
Minibatch accuracy: 43.8%
Validation accuracy: 75.8%
Minibatch loss at step 350: 0.576862
Minibatch accuracy: 81.2%
Validation accuracy: 78.3%
Minibatch loss at step 400: 1.054027
Minibatch accuracy: 68.8%
Validation accuracy: 69.6%
Minibatch loss at step 450: 0.827923
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.909654
Minibatch accuracy: 75.0%
Validation accuracy: 78.6%
Min

Validation accuracy: 86.4%
Minibatch loss at step 4650: 0.268097
Minibatch accuracy: 93.8%
Validation accuracy: 86.8%
Minibatch loss at step 4700: 0.517219
Minibatch accuracy: 81.2%
Validation accuracy: 86.8%
Minibatch loss at step 4750: 0.528690
Minibatch accuracy: 81.2%
Validation accuracy: 86.5%
Minibatch loss at step 4800: 0.330345
Minibatch accuracy: 93.8%
Validation accuracy: 87.0%
Minibatch loss at step 4850: 0.316650
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 4900: 0.493556
Minibatch accuracy: 81.2%
Validation accuracy: 86.8%
Minibatch loss at step 4950: 0.663409
Minibatch accuracy: 81.2%
Validation accuracy: 87.3%
Minibatch loss at step 5000: 1.264354
Minibatch accuracy: 81.2%
Validation accuracy: 86.8%
Minibatch loss at step 5050: 0.427286
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 5100: 0.268331
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 5150: 0.448715
Minibatch accuracy: 81.2%

Validation accuracy: 88.0%
Minibatch loss at step 9250: 0.207204
Minibatch accuracy: 93.8%
Validation accuracy: 88.5%
Minibatch loss at step 9300: 0.137608
Minibatch accuracy: 93.8%
Validation accuracy: 88.5%
Minibatch loss at step 9350: 0.141612
Minibatch accuracy: 100.0%
Validation accuracy: 88.1%
Minibatch loss at step 9400: 0.230423
Minibatch accuracy: 93.8%
Validation accuracy: 88.1%
Minibatch loss at step 9450: 0.113403
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
Minibatch loss at step 9500: 0.600491
Minibatch accuracy: 93.8%
Validation accuracy: 87.5%
Minibatch loss at step 9550: 0.161859
Minibatch accuracy: 93.8%
Validation accuracy: 88.1%
Minibatch loss at step 9600: 0.288967
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
Minibatch loss at step 9650: 0.888611
Minibatch accuracy: 75.0%
Validation accuracy: 87.9%
Minibatch loss at step 9700: 0.447308
Minibatch accuracy: 81.2%
Validation accuracy: 88.2%
Minibatch loss at step 9750: 0.004058
Minibatch accuracy: 100.

Validation accuracy: 88.3%
Minibatch loss at step 13850: 0.031893
Minibatch accuracy: 100.0%
Validation accuracy: 88.7%
Minibatch loss at step 13900: 0.729316
Minibatch accuracy: 81.2%
Validation accuracy: 88.1%
Minibatch loss at step 13950: 0.117386
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 14000: 0.113616
Minibatch accuracy: 93.8%
Validation accuracy: 88.7%
Minibatch loss at step 14050: 0.189164
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 14100: 0.290590
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 14150: 0.136811
Minibatch accuracy: 100.0%
Validation accuracy: 88.5%
Minibatch loss at step 14200: 0.122204
Minibatch accuracy: 100.0%
Validation accuracy: 88.8%
Minibatch loss at step 14250: 0.344337
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 14300: 0.024373
Minibatch accuracy: 100.0%
Validation accuracy: 88.8%
Minibatch loss at step 14350: 0.008947
Minibatch 

Validation accuracy: 88.8%
Minibatch loss at step 18450: 0.239293
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 18500: 0.904522
Minibatch accuracy: 75.0%
Validation accuracy: 89.0%
Minibatch loss at step 18550: 0.522570
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 18600: 0.475561
Minibatch accuracy: 87.5%
Validation accuracy: 89.1%
Minibatch loss at step 18650: 0.421878
Minibatch accuracy: 93.8%
Validation accuracy: 89.1%
Minibatch loss at step 18700: 0.289865
Minibatch accuracy: 93.8%
Validation accuracy: 89.1%
Minibatch loss at step 18750: 0.337495
Minibatch accuracy: 93.8%
Validation accuracy: 89.0%
Minibatch loss at step 18800: 0.155533
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 18850: 0.275402
Minibatch accuracy: 87.5%
Validation accuracy: 89.0%
Minibatch loss at step 18900: 0.437960
Minibatch accuracy: 87.5%
Validation accuracy: 88.9%
Minibatch loss at step 18950: 0.366500
Minibatch accu

The result is pretty good, however we still can improve it by introducing regularizations

In [20]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
beta_regul = 1e-3
dropout_rate = 0.5

graph = tf.Graph()

with graph.as_default():
    
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)
    
    # Input
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    layer3_weights = tf.Variable(tf.truncated_normal(
        [size3 * size3 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_hidden], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer5_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model our CNN
    def model(data):
        
        # C1 input 28 x 28
        conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
        hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
        # S1 input 24 x 24
        pool_1 = tf.nn.max_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        
        # C2 input 12 x 12
        conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='VALID')
        hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
        # S2 input 8 x 8
        pool_2 = tf.nn.max_pool(hidden_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        
        # FC3 input 4 x 4
        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden_3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        drop_1 = tf.nn.dropout(hidden_3,keep_prob=dropout_rate)
        
        #FC4
        
        hidden_4 = tf.nn.relu(tf.matmul(hidden_3, layer4_weights) + layer4_biases)
        drop_2 = tf.nn.dropout(hidden_4, keep_prob=dropout_rate)
        
        
        return tf.matmul(drop_2, layer5_weights) + layer5_biases
  
  # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [21]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.842023
Minibatch accuracy: 12.5%
Validation accuracy: 9.9%
Minibatch loss at step 50: 2.322712
Minibatch accuracy: 12.5%
Validation accuracy: 20.9%
Minibatch loss at step 100: 1.687874
Minibatch accuracy: 43.8%
Validation accuracy: 32.4%
Minibatch loss at step 150: 1.810240
Minibatch accuracy: 18.8%
Validation accuracy: 49.4%
Minibatch loss at step 200: 1.753553
Minibatch accuracy: 50.0%
Validation accuracy: 56.0%
Minibatch loss at step 250: 0.832970
Minibatch accuracy: 81.2%
Validation accuracy: 62.8%
Minibatch loss at step 300: 1.805330
Minibatch accuracy: 43.8%
Validation accuracy: 66.4%
Minibatch loss at step 350: 0.774989
Minibatch accuracy: 81.2%
Validation accuracy: 70.7%
Minibatch loss at step 400: 1.290612
Minibatch accuracy: 56.2%
Validation accuracy: 67.6%
Minibatch loss at step 450: 0.871428
Minibatch accuracy: 75.0%
Validation accuracy: 73.3%
Minibatch loss at step 500: 1.260114
Minibatch accuracy: 75.0%
Validation accuracy: 71.5%
Mi

Minibatch loss at step 4600: 0.364117
Minibatch accuracy: 87.5%
Validation accuracy: 84.1%
Minibatch loss at step 4650: 0.469418
Minibatch accuracy: 87.5%
Validation accuracy: 85.0%
Minibatch loss at step 4700: 0.794380
Minibatch accuracy: 87.5%
Validation accuracy: 83.9%
Minibatch loss at step 4750: 0.684898
Minibatch accuracy: 75.0%
Validation accuracy: 85.7%
Minibatch loss at step 4800: 0.238295
Minibatch accuracy: 93.8%
Validation accuracy: 85.6%
Minibatch loss at step 4850: 0.470656
Minibatch accuracy: 87.5%
Validation accuracy: 85.7%
Minibatch loss at step 4900: 0.371821
Minibatch accuracy: 81.2%
Validation accuracy: 85.5%
Minibatch loss at step 4950: 0.573379
Minibatch accuracy: 81.2%
Validation accuracy: 86.0%
Minibatch loss at step 5000: 0.771907
Minibatch accuracy: 81.2%
Validation accuracy: 85.3%
Test accuracy: 90.9%


Accuracy is worst. There are a few hyperparameters in the Net that can be tuned (dropput_rate, depths, lerning_rate etc...). The one can try to figure out which one work better.